In [1]:
%reset -f
import numpy as np
import time

import matplotlib.pyplot as plt
%run Functions.ipynb
%run Algorithms.ipynb
%run optSolver_OptiVisioners_Table.ipynb

In [2]:
# Define three class objects required in 'optSolver' and their attributes

class Problem:
    def __init__(self, name, x0, n, compute_f, compute_g, compute_H):
        self.name = name
        self.x0 = x0
        self.n = n
        self.compute_f = compute_f
        self.compute_g = compute_g
        self.compute_H = compute_H


class Method:
    def __init__(self, name, step_type, step_size):
        self.name = name
        self.step_type = step_type
        self.step_size = step_size


class Options:
    def __init__(self, term_tol=1e-6, max_iterations=10, c_1_ls=1e-4, c_2_ls=0.9, c_1_tr=0.25, c_2_tr=0.75, term_tol_CG=1e-4, max_iterations_CG=1000, memory_size=5, B=1e-6, r=0.2, tau=0.5):
        self.term_tol = term_tol
        self.max_iterations = max_iterations
        self.c_1_ls = c_1_ls
        self.c_2_ls = c_2_ls
        self.c_1_tr = c_1_tr
        self.c_2_tr = c_2_tr
        self.term_tol_CG = term_tol_CG
        self.max_iterations_CG = max_iterations_CG
        self.memory_size = memory_size
        self.B = B
        self.r = r
        self.tau = tau


# Define the options
options = Options(1e-6, 1e3)
optionsTR = Options(1e-4, 1e3)

In [3]:
# Define the problems
problems = []

np.random.seed(0)
# Problem 1 - Quadratic I
x0 = 20 * np.random.rand(10) - 10
problems.append(Problem('P1', x0, len(x0), P1_quad_10_10_func,
                P1_quad_10_10_grad, P1_quad_10_10_Hess))

# Problem 2 - Quadratic II
x0 = 20 * np.random.rand(10) - 10
problems.append(Problem('P2', x0, len(x0), P2_quad_10_1000_func,
                P2_quad_10_1000_grad, P2_quad_10_1000_Hess))

# Problem 3 - Quadratic III
x0 = 20 * np.random.rand(1000) - 10
problems.append(Problem('P3', x0, len(x0), P3_quad_1000_10_func,
                P3_quad_1000_10_grad, P3_quad_1000_10_Hess))

# Problem 4 - Quadratic IIII
x0 = 20 * np.random.rand(1000) - 10
problems.append(Problem('P3', x0, len(x0), P4_quad_1000_1000_func,
                P4_quad_1000_1000_grad, P4_quad_1000_1000_Hess))

# Problem 5 - Quartic I
theta = np.radians(70)  # convert angle to radians
x0 = np.array([np.cos(theta), np.sin(theta), np.cos(theta), np.sin(theta)])
problem = Problem('P5', x0, len(x0), P5_quartic_1_func,
                  P5_quartic_1_grad, P5_quartic_1_Hess)
problems.append(problem)

# Problem 6 - Quartic II
theta = np.radians(70)  # convert angle to radians
x0 = np.array([np.cos(theta), np.sin(theta), np.cos(theta), np.sin(theta)])
problem = Problem('P6', x0, len(x0), P6_quartic_2_func,
                  P6_quartic_2_grad, P6_quartic_2_Hess)
problems.append(problem)

# Problem 7 - Rosenbrock 2
x0 = np.array([-1.2, 1])
problem = Problem('P2', x0, len(x0), rosen2_func, rosen2_grad, rosen2_Hess)
problems.append(problem)

# Problem 8 - Rosenbrock 100
x0 = np.array([-1.2]+[1]*99)
problem = Problem('P8', x0, len(x0), rosen100_func,
                  rosen100_grad, rosen100_Hess)
problems.append(problem)

# Problem 9- Datafit
x0 = np.array([1, 1])
problem = Problem('P9', x0, len(x0), DataFit_2_func,
                  DataFit_2_grad, DataFit_2_Hess)
problems.append(problem)


# Problem 10- Exponential 10
x0 = np.array([-1.2]+[1]*99)
problem = Problem('P10', x0, len(x0), Exponential_10_func,
                  Exponential_10_grad, Exponential_10_Hess)
problems.append(problem)


# Problem 11- Exponential 100
x0 = np.array([1]+[0]*99)
problem = Problem('P11', x0, len(x0), Exponential_100_func,
                  Exponential_100_grad, Exponential_100_Hess)
problems.append(problem)


# Problem 12 Genhump
x0 = np.array([-506.2, 506.2, 506.2, 506.2, 506.2])
problem = Problem('P12', x0, len(x0), Genhumps_5_func,
                  Genhumps_5_grad, Genhumps_5_Hess)
problems.append(problem)

## Code to run and get all tables as dataframe

In [4]:
import numpy as np
import pandas as pd
global df, df1, df2, df3
columns = ['GD B', 'GD W', 'NW B', 'NW W', 'TR NW',
           'TR SR', 'BFGS B', 'BFGS W', 'DFP B', 'DFP W']

iterations = pd.DataFrame(columns=columns)
function = pd.DataFrame(columns=columns)
gradient = pd.DataFrame(columns=columns)
CPU = pd.DataFrame(columns=columns)


def run_config(i, j, config, df, df1, df2, df3):
    start_time = time.perf_counter()
    # Implement the algorithm on a problem
    x, f, x_list, f_list, c_list = optSolver_OptiVisioners_Table(
        config['problem'], config['method'], config['options'])
    end_time = time.perf_counter()  # stop measuring time

    total_time = end_time - start_time
    f_iters, g_iters = print_results(config, x, f, x_list, c_list, total_time)

    df.at[i, columns[j]] = len(x_list)

    df1.at[i, columns[j]] = f_iters

    df2.at[i, columns[j]] = g_iters

    df3.at[i, columns[j]] = total_time

    return df, df1, df2, df3


def print_results(config, x, f, x_list, c_list, total_time):
    print(
        f'Results for {config["method"].name} with step type {config["method"].step_type}')
    iters = len(x_list)
    if config['method'].step_type == 'Backtracking':
        f_iters = len(x_list) + np.sum(c_list)
        g_iters = len(x_list)
    elif config['method'].step_type == 'Wolfe' or config['method'].step_type == 'SR1':
        f_iters = len(x_list) + np.sum(c_list)
        g_iters = len(x_list) + np.sum(c_list)
    else:
        f_iters = len(x_list)
        g_iters = len(x_list)

    return f_iters, g_iters


for i, problem in enumerate(problems):
    configs = []
    configs.append({'problem': problem, 'options': options,
                   'method': Method('GradientDescent', 'Backtracking', 1)})
    configs.append({'problem': problem, 'options': options,
                   'method': Method('GradientDescent', 'Wolfe', 1)})
    configs.append({'problem': problem, 'options': options,
                   'method': Method('Newton', 'Backtracking', 1)})
    configs.append({'problem': problem, 'options': options,
                   'method': Method('Newton', 'Wolfe', 1)})
    configs.append({'problem': problem, 'options': optionsTR,
                   'method': Method('TRNewtonCG', 'TRN', 1)})
    configs.append({'problem': problem, 'options': optionsTR,
                   'method': Method('TRSR1CG', 'SR1', 1)})
    configs.append({'problem': problem, 'options': options,
                   'method': Method('BFGS', 'Backtracking', 1)})
    configs.append({'problem': problem, 'options': options,
                   'method': Method('BFGS', 'Wolfe', 1)})
    configs.append({'problem': problem, 'options': options,
                   'method': Method('DFP', 'Backtracking', 1)})
    configs.append({'problem': problem, 'options': options,
                   'method': Method('DFP', 'Wolfe', 1)})
    for j, config in enumerate(configs):
        iterations, function, gradient, CPU = run_config(
            i, j, config, iterations, function, gradient, CPU)

Results for GradientDescent with step type Backtracking
Results for GradientDescent with step type Wolfe
Results for Newton with step type Backtracking
Results for Newton with step type Wolfe
Results for TRNewtonCG with step type TRN
Results for TRSR1CG with step type SR1
Results for BFGS with step type Backtracking
Results for BFGS with step type Wolfe
Results for DFP with step type Backtracking
Results for DFP with step type Wolfe
Results for GradientDescent with step type Backtracking
Results for GradientDescent with step type Wolfe
Results for Newton with step type Backtracking
Results for Newton with step type Wolfe
Results for TRNewtonCG with step type TRN
Results for TRSR1CG with step type SR1
Results for BFGS with step type Backtracking
Results for BFGS with step type Wolfe
Results for DFP with step type Backtracking
Results for DFP with step type Wolfe
Results for GradientDescent with step type Backtracking
Results for GradientDescent with step type Wolfe
Results for Newton wi

## Number of Iterations 

In [6]:
iterations

,GD B,GD W,NW B,NW W,TR NW,TR SR,BFGS B,BFGS W,DFP B,DFP W
0,105,105,1,1,19,5,26,26,39,39
1,1000,1000,1,1,726,50,56,56,1000,1000
2,116,116,1,1,270,10,31,31,44,44
3,1000,1000,1,1,1000,50,352,352,1000,1000
4,2,2,1000,1000,1000,50,3,3,3,3
5,6,6,38,38,25,50,24,24,65,65
6,1000,1000,20,20,1000,32,33,29,47,417
7,42,42,4,4,3,5,112,112,108,108
8,530,554,6,6,6,12,14,14,21,21
9,12,12,258,258,140,50,4,4,4,4


## Number of Functions Evaluations

In [7]:
function

,GD B,GD W,NW B,NW W,TR NW,TR SR,BFGS B,BFGS W,DFP B,DFP W
0,105,105,1,1,19,5.0,26,26,39,39
1,1000,1000,1,1,726,50.0,56,56,1000,1000
2,116,116,1,1,270,10.0,31,31,44,44
3,1000,1000,1,1,1000,50.0,352,352,1000,1000
4,2,2,10483,10483,1000,50.0,3,3,3,3
5,87,87,38,38,25,50.0,86,86,126,126
6,9833,9773,27,27,1000,32.0,52,58,66,439
7,461,461,4,4,3,5.0,1011,1011,613,613
8,2907,3033,6,6,6,12.0,22,22,29,29
9,12,12,258,258,140,50.0,4,4,4,4


## Number of Gradient Evaluations

In [8]:
gradient

,GD B,GD W,NW B,NW W,TR NW,TR SR,BFGS B,BFGS W,DFP B,DFP W
0,105,105,1,1,19,5.0,26,26,39,39
1,1000,1000,1,1,726,50.0,56,56,1000,1000
2,116,116,1,1,270,10.0,31,31,44,44
3,1000,1000,1,1,1000,50.0,352,352,1000,1000
4,2,2,1000,10483,1000,50.0,3,3,3,3
5,6,87,38,38,25,50.0,24,86,65,126
6,1000,9773,20,27,1000,32.0,33,58,47,439
7,42,461,4,4,3,5.0,112,1011,108,613
8,530,3033,6,6,6,12.0,14,22,21,29
9,12,12,258,258,140,50.0,4,4,4,4


## CPU Time

In [9]:
CPU

,GD B,GD W,NW B,NW W,TR NW,TR SR,BFGS B,BFGS W,DFP B,DFP W
0,0.08624,0.112374,0.002572,0.002423,0.026873,0.011445,0.023888,0.032812,0.034199,0.043969
1,0.891842,1.1961,0.00237,0.002568,1.105844,0.121768,0.058939,0.074164,0.964585,1.279341
2,8.111899,10.569344,0.244622,0.256852,33.113684,1.913235,3.670112,4.366894,4.878808,5.736186
3,65.493146,86.97451,0.221787,0.242588,119.544492,9.656159,43.266387,49.154821,108.74414,130.082518
4,0.000211,0.000197,0.239499,0.575283,0.114093,0.00975,0.000409,0.000394,0.000498,0.000358
5,0.001582,0.003651,0.006942,0.005503,0.005041,0.011314,0.003247,0.005427,0.006817,0.009119
6,0.074797,0.188419,0.003182,0.002378,0.107288,0.004428,0.002884,0.003383,0.003485,0.029833
7,0.066547,0.268696,0.010805,0.009288,0.013486,0.018554,0.226391,0.736318,0.17109,0.464171
8,0.041405,0.132469,0.002296,0.001831,0.000966,0.002281,0.001546,0.002134,0.002024,0.002558
9,0.002476,0.003673,0.43003,0.45309,0.085784,0.042786,0.003239,0.00336,0.002891,0.003145
